We want interactive widgets in the notebook when plotting. Use the following magic command:

In [ ]:
%matplotlib notebook

### 1. Import temmeta and check out the contents of your file
you can use temmeta to print the contents as well as do some interactive visualisations of the dataset

In [ ]:
from temmeta import data_io as dio

In [ ]:
input_path = "data/sample_data.emd"
f = dio.EMDFile(input_path)
f.print_simple_structure()

In [ ]:
f.get_dataset("Image", "07a19a7f6e4248178f4cf2ab41c0bb07").plot_interactive()

### 2. We use the following tools to export the contents of the emd file and set up the config file for non-rigid registration
You will get messages where each dataset was exported to and which parameter file was created. You may get warnings that some image files can not be exported because they do not represent actual images.

In [ ]:
from jnrr import io_tools
from jnrr import processing

In [ ]:
import logging

In [ ]:
logger = logging.getLogger("Processing")
logger.setLevel(logging.INFO)

In [ ]:
paths = io_tools.extract_emd(input_path)

**The paths to the various files are returned by the `extract_emd` function as a dictionary. You will see that for each image dataset a different config file is created. So you can easily choose on which dataset you wish to run the calculation.**

In [ ]:
paths

**We don't need the emd file anymore, so it is best practice to close it**

In [ ]:
f.close()

**We can modify the config files with `read_config_file`, edit the dictionary values, and then save the result back out with `write_dict_to_config_file`. Let's say we want to limit the number of images that we want to use for the calculation to save time and resources. We will also edit the regularization parameter `lambda`. The different parameters will hopefully be better documented in the future**

In [ ]:
selected_config = paths["config_file_paths"][2]
conf = io_tools.read_config_file(selected_config)
conf

In [ ]:
conf["numTemplates"]="20"
conf["lambda"]="20"
io_tools.write_dict_to_config_file(selected_config, conf)

### 3. Run the non-rigid-registration
**Now we perform the calculation on a selected config file. This will calculate the several deformation fields in the images. For convenience, when it is done, the function returns the path to the folder where the results are stored.**

**!!! Note that this will only work on Linux systems !!!**

In [ ]:
!matchSeries {selected_config}

calculations_folder = processing.calculate_non_rigid_registration(selected_config)

### 4. Apply the deformations to selected datasets

**Now use the helper function to apply the deformations to the images and possibly the spectra. You must provide the path to the calculation folder. If you do not supply a path to an image folder, the one in the config file will be taken. One might wish to calculate deformations on one set of images, and apply them to a different set, hence why you have the option. A path to a spectrum dataset must be provided if you desire to apply them; by default none will be performed. The process of applying deformations to large spectrum maps can be quite slow. Check the help of the function to find out what is returned. The results are also stored as hyperspy datasets in a results folder.**

In [ ]:
from importlib import reload
reload(io_tools)
reload(processing)

In [ ]:
calculations_folder = "./data/nonrigid_results_002/"

In [ ]:
selected_image_folder = paths["image_folder_paths"][2]
selected_spectra_folder = paths["spectrum_folder_paths"][0]
results = processing.apply_deformations(calculations_folder, selected_image_folder, selected_spectra_folder)
averageUndeformed, averageDeformed, spectrumUndeformed, spectrumDeformed = results

### Visualize and work with the resulting datasets

In [ ]:
averageUndeformed.plot()

In [ ]:
averageDeformed.plot()

In [ ]:
spectrumUndeformed.plot_interactive()

In [ ]:
spectrumDeformed.plot_interactive()

**Check out the TEMMETA examples for further analysis techniques, or convert to a hyperspy object and continue your analysis with hyperspy.**